<a href="https://colab.research.google.com/github/Dminor7/ml-praticals-ldrp/blob/master/Consumer_Propensity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive for Colab Notebook


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Cloning Git Repository to Drive


In [4]:
!git clone https://github.com/Dminor7/ml-praticals-ldrp.git

fatal: destination path 'ml-praticals-ldrp' already exists and is not an empty directory.


Changing Directory

In [4]:
#!ls
%cd /content/ml-praticals-ldrp

/content/ml-praticals-ldrp


Importing Libraries For SQL Practice


In [5]:
from e_commerce.bigquery import BigQuery
from e_commerce.frame import profiler

In [28]:
ga_data = BigQuery("./service-account.json").get_data("""
        SELECT
        channelGrouping AS traffic_type,
        device.deviceCategory AS device_type,
        date,
        fullVisitorId,
        IF
        ( COUNTIF ( totals.transactions > 0
            AND totals.newVisits IS NULL ) > 0,
            1,
            0 ) AS had_buy_on_return_visit,
        SUM(totals.visits) AS visits,
        SUM(totals.pageviews) AS pageviews,
        SUM(totals.bounces) AS bounces,
        SUM(totals.transactions) AS transactions,
        SUM(totals.transactionRevenue)/1000000 AS revenue,
        ROUND((SUM(totals.transactionRevenue)/1000000)/COUNT( fullVisitorId),2) AS revenue_per_user,
        ROUND(SUM(totals.pageviews)/COUNT( fullVisitorId),2) AS pageviews_per_user,
        ROUND(SUM(totals.transactions)/COUNT( fullVisitorId),2) AS transactions_per_user,
        FROM
        `bigquery-public-data.google_analytics_sample.ga_sessions_*`
        WHERE
        _TABLE_SUFFIX BETWEEN '20160801'
        AND '20170801'
        GROUP BY
        channelGrouping,
        device_type,
        date,
        fullVisitorId
""")

In [29]:
#!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
ga_data_copy = ga_data
#df.head()
#df.dtypes
#df.describe()
#profiler(df)

Importing Libraries for Data Summarizing, Cleaning, Processing, Visualizing and Modeling

In [14]:
import os
import random 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

Reading Datafile (handling columns with JSON values)

In [10]:
import zipfile
# to convert json in df
import json 
# to normalize the json file
from pandas.io.json import json_normalize 
# Columns that have json format
columns = ['device', 'geoNetwork', 'totals', 'trafficSource'] 
dir_path = '/content/ml-praticals-ldrp/'
# p is a fractional number to skiprows and read just a random sample of the our dataset. 
p = 0.07 # *** In this case we will use 70% of data set *** #
#Code to transform the json format columns in table
def json_read(df):
    #joining the [ path + df received]
    data_frame = dir_path + df
    #Importing the dataset
    zf = zipfile.ZipFile(data_frame+".zip") 
    
    df = pd.read_csv(zf.open('train_v2.csv'), 
                     converters={column: json.loads for column in columns}, # loading the json columns properly
                     dtype={'fullVisitorId': 'str'}, # transforming this column to string
                     skiprows=lambda i: i>0 and random.random() > p)# Number of rows that will be imported randomly
    
    for column in columns: #loop to finally transform the columns in data frame
        #It will normalize and set the json to a table
        column_as_df = json_normalize(df[column]) 
        # here will be set the name using the category and subcategory of json columns
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns] 
        # after extracting the values, let drop the original columns
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
        
    # Printing the shape of dataframes that was imported  
    print(f"Loaded {os.path.basename(data_frame)}. Shape: {df.shape}")
    return df 


In [15]:
%%time 
# %%time is used to calculate the timing of code chunk execution #

df_train = json_read("train_v2.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


Loaded train_v2.csv. Shape: (119907, 59)
CPU times: user 2min 28s, sys: 4.89 s, total: 2min 33s
Wall time: 2min 34s


In [16]:
df_train.head()

,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.timeOnSite,totals.newVisits,totals.sessionQualityDim,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,totals.bounces,trafficSource.referralPath,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.keyword,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adContent
0,Referral,"[{'index': '4', 'value': 'EMEA'}]",20171016,8349655975937271469,"[{'hitNumber': '1', 'time': '0', 'hour': '4', ...",Not Socially Engaged,1508152478,1,1508152478,Chrome,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Europe,Northern Europe,United Kingdom,England,London,London,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,2,2,9,1,1,NaN,NaN,NaN,NaN,/a/google.com/nest-vision/dropcam-field-tester...,(not set),sites.google.com,referral,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171016,7058083259448191057,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,1508174145,1,1508174145,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,Northern America,Canada,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,2,2,1266,1,1,NaN,NaN,NaN,NaN,NaN,(not set),google,organic,not available in demo dataset,(not provided),NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,[],20171016,9630953897602496525,"[{'hitNumber': '1', 'time': '0', 'hour': '19',...",Not Socially Engaged,1508206555,2,1508206555,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,South America,Peru,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,utp.edu.pe,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,2,2,111,NaN,

Summarizing Data with pandas profiler


In [17]:
profiler(df_train)

Summarize dataset:   0%|          | 0/67 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Traditional Approach to Summarize data

In [18]:
def describe_data(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values
    
    return summary

In [19]:
describe_data(df_train)

Dataset Shape: (119907, 59)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value
0,channelGrouping,object,0,8,Referral,Organic Search,Organic Search
1,customDimensions,object,0,6,"[{'index': '4', 'value': 'EMEA'}]","[{'index': '4', 'value': 'North America'}]",[]
2,date,int64,0,638,20171016,20171016,20171016
3,fullVisitorId,object,0,114874,8349655975937271469,7058083259448191057,9630953897602496525
4,hits,object,0,111230,"[{'hitNumber': '1', 'time': '0', 'hour': '4', ...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...","[{'hitNumber': '1', 'time': '0', 'hour': '19',..."
5,socialEngagementType,object,0,1,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged
6,visitId,int64,0,119608,1508152478,1508174145,1508206555
7,visitNumber,int64,0,217,1,1,2
8,visitStartTime,int64,0,119616,1508152478,1508174145,1508206555
9,device.browser,object,0,41,Chrome,Chrome,Chrome


Analyzing Columns with Null values

In [24]:
def Null_Count(df):
    df_null = df.isnull().sum().sort_values(ascending = False).rename('Null').reset_index()

    null_count = df_null['Null']
    null_percent = (null_count * 100) / (df.shape[0])

    df_null = pd.concat([df_null['index'],null_count,null_percent], axis=1, keys=['Column','Null_Count','Null_Percent'])

    return df_null[df_null['Null_Count'] != 0]



Null_Count(df_train)

,Column,Null_Count,Null_Percent
0,totals.transactionRevenue,118600,98.909989
1,totals.totalTransactionRevenue,118600,98.909989
2,totals.transactions,118598,98.908321
3,trafficSource.adContent,115369,96.215400
4,trafficSource.adwordsClickInfo.adNetworkType,114626,95.595753
5,trafficSource.adwordsClickInfo.slot,114626,95.595753
6,trafficSource.adwordsClickInfo.page,114626,95.595753
7,trafficSource.adwordsClickInfo.isVideoAd,114626,95.595753
8,trafficSource.adwordsClickInfo.gclId,114614,95.585746
9,trafficSource.isTrueDirect,82441,68.754118


Processing Null values

In [25]:
def fill_na(df):   
    df['totals.pageviews'].fillna(1, inplace=True)
    df['totals.newVisits'].fillna(0, inplace=True)
    df['totals.bounces'].fillna(0, inplace=True) 
    df["totals.transactionRevenue"].fillna(0.0, inplace=True)
    
    # Changing datatypes from object to desired ones
    df['totals.pageviews'] = df['totals.pageviews'].astype(int)
    df['totals.newVisits'] = df['totals.newVisits'].astype(int)
    df['totals.bounces'] = df['totals.bounces'].astype(int)
    df["totals.transactionRevenue"] = df["totals.transactionRevenue"].astype(float)

    return df

df_train = fill_na(df_train)

Processing Date/Time

In [26]:
from datetime import datetime

# This function is to extract date features
def date_process(df):
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d") # seting the column as pandas datetime
    df["weekday"] = df['date'].dt.weekday #extracting week day
    df["day"] = df['date'].dt.day # extracting day
    df["month"] = df['date'].dt.month # extracting day
    df["year"] = df['date'].dt.year # extracting day
    df['visitHour'] = (df['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))).astype(int)
    
    return df

df_train = date_process(df_train)

Removing Constant Columns

In [27]:
constant_columns = [col for col in df_train.columns if df_train[col].nunique() == 1]
print(f'Columns : {constant_columns}, \n Num of Columns : {len(constant_columns)}')

df_train.drop(constant_columns, axis=1, inplace=True)

Columns : ['socialEngagementType', 'device.browserVersion', 'device.browserSize', 'device.operatingSystemVersion', 'device.mobileDeviceBranding', 'device.mobileDeviceModel', 'device.mobileInputSelector', 'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName', 'device.flashVersion', 'device.language', 'device.screenColors', 'device.screenResolution', 'geoNetwork.cityId', 'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.networkLocation', 'totals.visits', 'trafficSource.adwordsClickInfo.criteriaParameters', 'trafficSource.isTrueDirect', 'trafficSource.adwordsClickInfo.isVideoAd'], 
 Num of Columns : 21


Droping Null Columns

In [28]:
Null_Count(df_train)

In [ ]:
df_train.drop(list(Null_Count(df_train)['Column']), axis=1,inplace=True)

Exploring data after Cleaning and Processing

In [29]:
describe_data(df_train)

Dataset Shape: (119907, 32)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value
0,channelGrouping,object,0,8,Referral,Organic Search,Organic Search
1,customDimensions,object,0,6,"[{'index': '4', 'value': 'EMEA'}]","[{'index': '4', 'value': 'North America'}]",[]
2,date,datetime64[ns],0,638,2017-10-16 00:00:00,2017-10-16 00:00:00,2017-10-16 00:00:00
3,fullVisitorId,object,0,114874,8349655975937271469,7058083259448191057,9630953897602496525
4,hits,object,0,111230,"[{'hitNumber': '1', 'time': '0', 'hour': '4', ...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...","[{'hitNumber': '1', 'time': '0', 'hour': '19',..."
5,visitId,int64,0,119608,1508152478,1508174145,1508206555
6,visitNumber,int64,0,217,1,1,2
7,visitStartTime,int64,0,119616,1508152478,1508174145,1508206555
8,device.browser,object,0,41,Chrome,Chrome,Chrome
9,device.operatingSystem,object,0,19,Macintosh,Windows,Windows


Reinitialized existing Git repository in /content/ml-praticals-ldrp/.git/


In [40]:
!git init
!git config --global user.email "himalya.bachwani@gmail.com"
!git config --global user.name "Himalya"
!git add .
!git commit -m "Till Cleaning and Processing"
!git push 

Reinitialized existing Git repository in /content/ml-praticals-ldrp/.git/
On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address


fatal: could not read Username for 'https://github.com': No such device or address
